# 2-4-3 LoRA

[LoRA: Low-Rank Adaptation of Large Language Models](https://arxiv.org/abs/2106.09685)

## 0 章节目标

* 理解 LoRA 的基本思想
* 学习如何使用 Huggingface 的代码微调自己的模型
* 了解 LoRA 在其他 LLM 中的应用

## 1 问题背景

为什么仅用几千条样本就能将一个数十亿参数的模型微调得比较好？以往的一些结果（如 [Qin et al.](https://arxiv.org/abs/2012.13255) 和 [Aghajanyan et al.](https://arxiv.org/abs/2012.13255)）显示，尽管预训练模型的参数量很大，但每个下游任务对应的本征维度（Intrinsic Dimension）并不大，换句话说，理论上我们可以微调大模型中很小一部分的参数，就能很好地适应下游任务。

何为本征维度？本征维度是指达到任务期望性能所需的最小参数量。如果把大模型看作是将数据映射到高维空间进行处理的一种模型，那么某一个具体的细分任务，可能只需要用到这一高维空间的某个子空间。我们无法找到解决这些问题的最佳的子空间，但可以手动选取某些参数作为子空间。当我们对该子空间中的参数进行微调时，能够达到全量参数微调的一定水平，如 [Aghajanyan et al.](https://arxiv.org/abs/2012.13255) 论文中选择的 $90\%$ 的性能，那么这一子空间满足该下游任务的要求。所有符合要求的子空间中，维度最小的那个子空间对应的维度为该任务的本征维度，记为 $d$。

假定模型的全量参数为 $\theta^D$，其中本征维度上的参数为 $\theta^d$，那么针对某个特定下游任务的微调可以表示为：

$$\theta^D=\theta^D_0+\theta^d M$$

其中矩阵 $M$ 将本征维度上的参数向量 $\theta^d\in\mathbb{R}^d$ 映射到全量参数空间 $\mathbb{R}^D$ 中。这是大模型信息压缩的观点。

LoRA 的思想正是基于这样的观点，认为模型在微调 adaptation 的时候也具有这样的内在秩，参数的增量可以被低秩分解，只需要优化低秩分解的矩阵即可。

## 2 微调方法

对于预训练的参数矩阵 $W_0^\mathbb{R}^{m\times n}$，LoRA 不直接微调 $W_0$，而是对增量作低秩分解：

$$W = W_0 + A B,\quad A\in\mathbb{R}^{m\times r},B\in\mathbb{R}^{r\times n}$$

其中 $A,\,B$ 其一使用全零初始化。由于本征维度很小的结论，低秩矩阵的 $r$ 可以取的很小，甚至为 $1$，因此需要被优化的参数量大大降低了。

![](https://img-blog.csdnimg.cn/img_convert/06b30661c111c0cae25adbaa9c0e3819.png#averageHue=#eb9245&clientId=ue4258a53-1a5a-4&from=paste&height=376&id=u6bc01447&originHeight=453&originWidth=496&originalType=binary&ratio=2&rotation=0&showTitle=false&size=25068&status=done&style=none&taskId=u1f08ffa2-cd94-46bf-baea-46a2cf18e48&title=&width=412)

LoRA 在微调预训练参数的子集这种大模型微调范式的基础上进一步减少了约束，不要求在微调过程中累积梯度更新的权重矩阵具有满秩，而随着 LoRA 可微调参数逐渐增大，直至微调全部参数，LoRA 低秩矩阵的秩也收敛于预训练模型参数矩阵的秩。换句话说，随着可训练参数的增加，LoRA 收敛于全量参数模型，而基于 Adapter 的微调方案收敛于 MLP，基于 Prefix Tuning 的方案可接收的输入序列则越来越小。

需要注意的是，LoRA 和其他很多微调方案一样，只是降低了权重，但没有降低计算量。关于这一点，我们只需要观察低秩矩阵 $A,\,B$ 的梯度：

$$\frac{\partial \mathcal{L}}{\partial A} = \frac{\partial \mathcal{L}}{\partial W} B^{\top},\quad \frac{\partial \mathcal{L}}{\partial B} = A^{\top}\frac{\partial \mathcal{L}}{\partial W}$$

在训练过程中，LoRA 更新所用的梯度 $\frac{\partial \mathcal{L}}{\partial A},\,\frac{\partial \mathcal{L}}{\partial B}$ 来自全量更新的梯度 $\frac{\partial \mathcal{L}}{\partial W}$，因此从理论上来说 LoRA 的计算量没有减少。但是 LoRA 与其他参数高效的微调方案一样，都可以通过冻结模型大部分层、以及更新参数量减少所带来的多卡通信效率提升来提高实际上的训练速度。与 Adapter、Prefix-Tuning 等方案类似地，LoRA 在不同任务之间切换的成本同样很低，对于几十 GB 的大模型来说，LoRA 的低秩矩阵模块可能只有几 MB。LoRA 的独有优势如上面所说，是一种更加直观，更接近全量微调的微调方式。此外在推理过程中，$AB$ 可以视为单个矩阵，这也可以减少推理成本。

## 3 LoRA 在 Stable Diffusion 中的应用

尽管 LoRA 的论文在 NLP 中的大模型基础上进行，但由于其实现的原理更加接近全量参数模型，而不像 Adapter 那样适用于某一类下游任务，或者像 Prefix-Tuning 那样与 token 这种文本相关的概念结合比较深，因此 LoRA 同样可以用于微调图像领域的大模型，就比如 Stable Diffusion。与原论文只更新 Self Attention 的参数类似，微调 Stable Diffusion 时 LoRA 一般应用于图文相似度模块的交叉注意力层。

Diffusers 提供了一个 LoRA 的微调脚本，可以借助这一脚本在无需量化技巧的情况下，在 12 GB 以内的 GPU 上微调 Stable Diffusion，并且产生的 LoRA 模块文件只有大约 3 MB。Diffusers 在 GitHub 中的最新的微调代码调用了 SNR，相关的代码只在 `diffusers` 库的开发版本，也就是从 GitHub 直接克隆下来的版本中存在。使用 `pip install diffusers` 安装的 `diffusers` 需要使用相匹配的[代码](https://github.com/huggingface/diffusers/blob/v0.21.4/examples/text_to_image/train_text_to_image_lora.py)。

Diffusers 关于 LoRA 的核心代码包括 `LoRALinearLayer` 和 `LoRAAttnProcessor`。

In [ ]:
from diffusers.models.attention_processor import Attention, AttnProcessor
from torch import nn


class LoRALinearLayer(nn.Module):
    def __init__(self, in_features, out_features, rank=4, network_alpha=None, device=None, dtype=None):
        super().__init__()

        self.down = nn.Linear(in_features, rank, bias=False, device=device, dtype=dtype)
        self.up = nn.Linear(rank, out_features, bias=False, device=device, dtype=dtype)
        self.network_alpha = network_alpha
        self.rank = rank
        self.out_features = out_features
        self.in_features = in_features

        nn.init.normal_(self.down.weight, std=1 / rank)
        nn.init.zeros_(self.up.weight)

    def forward(self, hidden_states):
        orig_dtype = hidden_states.dtype
        dtype = self.down.weight.dtype

        down_hidden_states = self.down(hidden_states.to(dtype))
        up_hidden_states = self.up(down_hidden_states)

        if self.network_alpha is not None:
            up_hidden_states *= self.network_alpha / self.rank

        return up_hidden_states.to(orig_dtype)


class LoRAAttnProcessor(nn.Module):
    def __init__(self, hidden_size, cross_attention_dim=None, rank=4, network_alpha=None, **kwargs):
        super().__init__()

        self.hidden_size = hidden_size
        self.cross_attention_dim = cross_attention_dim
        self.rank = rank

        q_rank = kwargs.pop("q_rank", None)
        q_hidden_size = kwargs.pop("q_hidden_size", None)
        q_rank = q_rank if q_rank is not None else rank
        q_hidden_size = q_hidden_size if q_hidden_size is not None else hidden_size

        v_rank = kwargs.pop("v_rank", None)
        v_hidden_size = kwargs.pop("v_hidden_size", None)
        v_rank = v_rank if v_rank is not None else rank
        v_hidden_size = v_hidden_size if v_hidden_size is not None else hidden_size

        out_rank = kwargs.pop("out_rank", None)
        out_hidden_size = kwargs.pop("out_hidden_size", None)
        out_rank = out_rank if out_rank is not None else rank
        out_hidden_size = out_hidden_size if out_hidden_size is not None else hidden_size

        self.to_q_lora = LoRALinearLayer(q_hidden_size, q_hidden_size, q_rank, network_alpha)
        self.to_k_lora = LoRALinearLayer(cross_attention_dim or hidden_size, hidden_size, rank, network_alpha)
        self.to_v_lora = LoRALinearLayer(cross_attention_dim or v_hidden_size, v_hidden_size, v_rank, network_alpha)
        self.to_out_lora = LoRALinearLayer(out_hidden_size, out_hidden_size, out_rank, network_alpha)

    def __call__(self, attn: Attention, hidden_states, *args, **kwargs):
        attn.to_q.lora_layer = self.to_q_lora.to(hidden_states.device)
        attn.to_k.lora_layer = self.to_k_lora.to(hidden_states.device)
        attn.to_v.lora_layer = self.to_v_lora.to(hidden_states.device)
        attn.to_out[0].lora_layer = self.to_out_lora.to(hidden_states.device)

        attn._modules.pop("processor")
        attn.processor = AttnProcessor()
        return attn.processor(attn, hidden_states, *args, **kwargs)

### 安装必要的库

In [ ]:
!pip install accelerate diffusers transformers

### 运行脚本

In [ ]:
!export MODEL_NAME="stabilityai/stable-diffusion-2-1"
!export OUTPUT_DIR="./lora/pokemon"
!export DATASET_NAME="lambdalabs/pokemon-blip-captions"

!accelerate launch --mixed_precision="fp16" train_text_to_image_lora.py \
    --pretrained_model_name_or_path=${MODEL_NAME} \
    --dataset_name=$DATASET_NAME --caption_column="text" \
    --resolution=512 --random_flip \
    --train_batch_size=1 \
    --num_train_epochs=100 --checkpointing_steps=5000 \
    --learning_rate=1e-04 --lr_scheduler="constant" --lr_warmup_steps=0 \
    --seed=42 \
    --output_dir="sd-pokemon-model-lora" \
    --validation_prompt="cute dragon creature"

### 推理

训练结束后使用 LoRA 的结果，只需要在基础模型的基础上加载 LoRA 权重

In [ ]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

model_name = "stabilityai/stable-diffusion-2-1"
lora_dir = "./lora/pokemon"

pipe = StableDiffusionPipeline.from_pretrained(model_name, torch_dtype=torch.float16)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.unet.load_attn_procs(lora_dir)
pipe.to("cuda")

image = pipe("Green pokemon with menacing face", num_inference_steps=25).images[0]
image.save("green_pokemon.png")

### 利用 LoRA 进行 DreamBoothing

DreamBooth 向模型中插入新的概念 [V]，与 LoRA 并不冲突。因此这意味着我们可以利用 LoRA 来微调 DreamBooth，来替代原本的 Fine-Tuning，提升训练速度并不再需要保存整个模型的参数。一个可以运行的脚本与[代码](https://github.com/huggingface/diffusers/blob/v0.21.4/examples/dreambooth/train_dreambooth_lora.py)同样可以在 Diffusers 中找到。

In [ ]:
!export MODEL_NAME="stabilityai/stable-diffusion-2-1"
!export INSTANCE_DIR="dog"
!export OUTPUT_DIR="./lora/dog"

!accelerate launch train_dreambooth_lora.py \
    --pretrained_model_name_or_path=$MODEL_NAME \
    --instance_data_dir=$INSTANCE_DIR \
    --output_dir=$OUTPUT_DIR \
    --instance_prompt="a photo of sks dog" \
    --resolution=512 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --checkpointing_steps=100 \
    --learning_rate=1e-4 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps=500 \
    --validation_prompt="A photo of sks dog in a bucket" \
    --validation_epochs=50 \
    --seed="0"